In [18]:
echo "# Q1_Jeff" >> README.md
git init
git add README.md
git commit -m "first commit"
git branch -M main
git remote add origin https://github.com/alvaroforner/Q1_Jeff.git
git push -u origin main

SyntaxError: invalid syntax (Temp/ipykernel_18104/1938379535.py, line 1)

In [1]:
#Al no disponer de entorno donde poder ejecutar directamente en SQL he adaptado el código para poder ejecutarlo sobre Python:
import pandas as pd
import os
import duckdb

In [4]:
#Se caergan las dos tablas. Table_A contiene los usuarios y el Signup_Timestamp y Table_B contiene usuarios y Order_Timestamp
df_a = pd.read_excel("Table_A.xlsx", header = 0)
df_b = pd.read_excel("Table_B.xlsx", header = 0)

In [5]:
#Nos aseguramos de que solo estamos registrando para el análisis la primera compra que hace el usuario en la tabla B
df_b2=duckdb.query("Select user_id, order_timestamp from (\
             Select *,\
            ROW_NUMBER() over(partition by user_id order by order_timestamp ASC) as rn\
        from df_b) as d  \
where d.rn =1\
;\
  ").df()

In [7]:
df_b2

,user_id,order_timestamp
0,a6,2022-05-17
1,a8,2022-05-17
2,a4,2022-05-08
3,a2,2022-05-03
4,a5,2022-05-08
5,a13,2022-07-04
6,a17,2022-06-30
7,a14,2022-06-27
8,a9,2022-06-30
9,a16,2022-06-29


In [8]:
#Generamos a partir de la tabla A un dataframe que contiene el número de la semana correspondiente al momento en que el usuario se registró
#además, añadimos el año del registro por si hubiera un cliente que se registró y realizó su primer pedido en años distintos.
#llamamos a eta tabla Cohort_Items.
cohort_items = duckdb.query("  select A.user_id,extract(week from signup_timestamp) as cohort_week ,extract(year from signup_timestamp) as cohort_year, signup_timestamp \
  from df_a A \
    group by 1, 2, 3, 4 \
                  ").df()

In [9]:
cohort_items

,user_id,cohort_week,cohort_year,signup_timestamp
0,a1,17,2021,2021-05-01
1,a2,17,2021,2021-05-02
2,a3,18,2021,2021-05-03
3,a4,18,2021,2021-05-04
4,a5,18,2021,2021-05-05
5,a6,18,2021,2021-05-06
6,a7,18,2021,2021-05-07
7,a8,18,2021,2021-05-08
8,a9,18,2021,2021-05-09
9,a10,19,2021,2021-05-10


In [10]:
#Generamos a partir de la tabla B (Ordenes) un dataframe que contiene el número de la semana en que el usuario realizó su primera orden.
#y llamamos a la tabla user_activities.
user_activities= duckdb.query("  select A.user_id, datediff('WEEK', c.signup_timestamp, a.order_timestamp) as week_number \
  from df_b2 A \
  left join cohort_items C ON A.user_id = C.user_id \
  group by 1, 2 \
                  ").df()

In [11]:
user_activities

,user_id,week_number
0,a6,53
1,a8,53
2,a4,53
3,a2,52
4,a5,53
5,a13,59
6,a17,59
7,a14,58
8,a9,60
9,a16,58


In [12]:
#Generamos la tabla Cohort_Saiz con el agrupamiento de usuarios según la semana del año en que se registraron.
cohort_size= duckdb.query("select cohort_week, cohort_year, count(1) as num_users \
  from cohort_items \
  group by 1, 2\
  order by 1, 2 ").df()

In [13]:
cohort_size

,cohort_week,cohort_year,num_users
0,17,2021,2
1,18,2021,7
2,19,2021,7
3,20,2021,1


In [14]:
#Generamos la tabla Retention_Table, que contiene el bote de registros tras el cruce entre las tablas Activities y Cohort_Items,
#de este modo nos traemos el número de la semana en que se registro el grupo de usuarios, y el número de semanas que tardó el grupo
#en realizar la primera orden.
retention_table= duckdb.query("select C.cohort_week, C.cohort_year, A.week_number, count(1) as num_users\
  from user_activities A\
  left join cohort_items C ON A.user_id = C.user_id\
  group by 1, 2, 3").df()

In [15]:
retention_table

,cohort_week,cohort_year,week_number,num_users
0,18,2021,53,5
1,17,2021,52,2
2,19,2021,59,1
3,20,2021,59,1
4,19,2021,58,3
5,18,2021,60,1
6,19,2021,60,3
7,18,2021,52,1


In [16]:
#Para finalizar, añadimos el porcentaje de usuarios que corresponde a cada week number en funcion del cohort week.
final= duckdb.query("select B.cohort_week, B.cohort_year, S.num_users as total_users, B.week_number, B.num_users::float * 100 / S.num_users as percentage \
from retention_table B \
left join cohort_size S ON B.cohort_week = S.cohort_week \
where B.cohort_week IS NOT NULL \
order by 1,2, 3 ").df()

In [17]:
final

,cohort_week,cohort_year,total_users,week_number,percentage
0,17,2021,2,52,100.000000
1,18,2021,7,53,71.428574
2,18,2021,7,60,14.285714
3,18,2021,7,52,14.285714
4,19,2021,7,59,14.285714
5,19,2021,7,58,42.857143
6,19,2021,7,60,42.857143
7,20,2021,1,59,100.000000
